In [2]:
import re
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from keras.models import Sequential,load_model
from keras.layers import Dense,Embedding
from keras.layers import LSTM,Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from sklearn.model_selection import train_test_split
import fasttext
import os
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
import fasttext.util
from gensim.models import KeyedVectors


Using TensorFlow backend.


In [6]:
model = KeyedVectors.load_word2vec_format('/Users/askvyas/Documents/wiki-news-300d-1M.vec')
print (model.most_similar('desk'))



[('desks', 0.7923153638839722), ('Desk', 0.6869951486587524), ('desk.', 0.6602819561958313), ('desk-', 0.6187258958816528), ('credenza', 0.5955315828323364), ('roll-top', 0.5875717401504517), ('rolltop', 0.5837830305099487), ('bookshelf', 0.5758029222488403), ('Desks', 0.5755287408828735), ('sofa', 0.5617446899414062)]


In [7]:
#Cleaning

data=pd.read_csv('offensiveMemes.csv')
data.text_corrected=data.text_corrected.astype(str)
data["text_corrected"]= data["text_corrected"].str.lower() 

data=data.sample(frac=1).reset_index(drop=True)


data['text_corrected'] = data['text_corrected'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))


In [8]:
a=data[data.offensive=='offensive']

In [9]:
a_n=a.sample(200).reset_index() 

ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
data_f=pd.DataFrame(data, columns = ['text_corrected', 'offensive'])

data_f = data_f.rename(columns={"text_corrected":"text", "offensive":"label"})


data_f['label'] = '__label__' + data_f['label'].astype(str)
data_f.to_csv('final.txt', sep='\t', index = False, header = False)


training_data_path=os.getcwd()+'/final.txt'
print(training_data_path)

In [ ]:
a=data_f[data_f['label']=='__label__not_offensive']
b=data_f[data_f['label']=='__label__very_offensive']
c=data_f[data_f['label']=='__label__slight']
d=data_f[data_f['label']=='__label__hateful_offensive']


data_v=a.sample(200)
data_v=data_v.append(b.sample(200))
data_v=data_v.append(c.sample(200))
data_v=data_v.append(d.sample(200))
data_v.to_csv('valid.txt', sep='\t', index = False, header = False)
validation_data_path=os.getcwd()+'/valid.txt'
print(validation_data_path)


In [ ]:
hyper_params = {"lr": 0.01,
                        "epoch": 20,
                        "wordNgrams": 2,
                        "dim": 100,
               "pretrainedVectors":'wiki-news-300d-1M.vec'
               }


model = fasttext.train_supervised(input=training_data_path, **hyper_params)



In [ ]:
        result = model.test(training_data_path)
        validation = model.test(training_data_path)
        
        # DISPLAY ACCURACY OF TRAINED MODEL
        text_line = str(hyper_params) + ",accuracy:" + str(result[1])  + ",validation:" + str(validation[1]) + '\n' 
        print(text_line)


In [ ]:
#model.quantize()

In [ ]:
        validation = model.test(validation_data_path)
        
        # DISPLAY ACCURACY OF TRAINED MODEL
        text_line = str(hyper_params) + ",accuracy:" + str(result[1])  + ",validation:" + str(validation[1]) + '\n' 
        print(text_line)


In [ ]:
data['offensive'].value_counts().sort_index().plot.bar()

In [ ]:
model.predict("sorry there are only two genders")